In [1]:
import numpy as np
import pandas as pd
import pickle
from fforma import *

### Testing data

In [2]:
monthly_m3 = pd.read_csv('data/data_m3/dataM3.csv').query('freq=="M"')

In [3]:
ids_monthly_m3 = monthly_m3['id'].unique()

In [4]:
list_monthly_m3 = [monthly_m3.query('(id==@idx) & (type=="train")')['value'].to_numpy() for idx in ids_monthly_m3]

In [5]:
list_monthly_m3_test = [monthly_m3.query('(id==@idx) & (type=="test")')['value'].to_numpy() for idx in ids_monthly_m3]

In [6]:
len(list_monthly_m3)

1428

In [7]:
len(list_monthly_m3_test)

1428

### Model parameters 

In [8]:
h_forecast = len(list_monthly_m3_test[10]) 
h_forecast

18

In [9]:
frequency = 12 # Monthly data

### Reading models 

In [14]:
models = ['fforma_'+str(n) for n in [1000]]#, 9600, 14400]]
models

['fforma_1000']

In [45]:
fforma_models = []
for str_model in models:
    with open(str_model, 'rb') as read_file:
        fforma_models.append(pickle.load(read_file))

### Making predictions 

In [46]:
preds = [fforma.predict(h_forecast, list_monthly_m3, frequency) for fforma in fforma_models]

/home/fede/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters.py:725: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
/home/fede/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters.py:725: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
/home/fede/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters.py:725: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
/home/fede/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters.py:725: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
/home/fede/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters.py:725: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)
/home/fede/.local/lib/python3.6/site-packages/statsmodels/tsa/holtwinters.py:725: ConvergenceWarning: Optimization faile

ValueError: feature_names mismatch: ['diff1_acf1', 'diff1_acf10', 'entropy', 'seas_pacf', 'garch_r2', 'diff2_acf10', 'nperiods', 'frequency', 'seas_acf1', 'series_length', 'e_acf1', 'x_acf1', 'flat_spots', 'diff2_acf1', 'crossing_points', 'trend', 'x_pacf5', 'curvature', 'arch_lm', 'arch_acf', 'linearity', 'lumpiness', 'garch_acf', 'e_acf10', 'diff1x_pacf5', 'x_acf10', 'spike', 'seasonal_period', 'diff2x_pacf5', 'stability', 'arch_2'] ['diff2_acf10', 'curvature', 'e_acf10', 'x_pacf5', 'lumpiness', 'e_acf1', 'series_length', 'x_acf10', 'diff1_acf10', 'nperiods', 'garch_acf', 'crossing_points', 'arch_acf', 'diff2_acf1', 'x_acf1', 'diff1_acf1', 'diff1x_pacf5', 'spike', 'flat_spots', 'seasonal_period', 'arch_2', 'seas_acf1', 'arch_lm', 'linearity', 'frequency', 'entropy', 'garch_r2', 'seas_pacf', 'stability', 'diff2x_pacf5', 'trend']

In [47]:
preds

NameError: name 'preds' is not defined

In [39]:
tsf_ = tsfeatures(list_monthly_m3, 12, parallel=True)

/home/fede/.local/lib/python3.6/site-packages/entropy/entropy.py:177: RuntimeWarning: divide by zero encountered in log2
  se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
/home/fede/.local/lib/python3.6/site-packages/entropy/entropy.py:177: RuntimeWarning: divide by zero encountered in log2
  se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
/home/fede/.local/lib/python3.6/site-packages/entropy/entropy.py:177: RuntimeWarning: divide by zero encountered in log2
  se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
/home/fede/.local/lib/python3.6/site-packages/entropy/entropy.py:177: RuntimeWarning: invalid value encountered in multiply
  se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
/home/fede/.local/lib/python3.6/site-packages/entropy/entropy.py:177: RuntimeWarning: invalid value encountered in multiply
  se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
/home/fede/.local/lib/python3.6/site-packages/entropy/entropy.py:177: RuntimeWarning: invalid value encountered in

In [40]:
tsf_.columns

Index(['diff2_acf10', 'curvature', 'e_acf10', 'x_pacf5', 'lumpiness', 'e_acf1',
       'series_length', 'x_acf10', 'diff1_acf10', 'nperiods', 'garch_acf',
       'crossing_points', 'arch_acf', 'diff2_acf1', 'x_acf1', 'diff1_acf1',
       'diff1x_pacf5', 'spike', 'flat_spots', 'seasonal_period', 'arch_2',
       'seas_acf1', 'arch_lm', 'linearity', 'frequency', 'entropy', 'garch_r2',
       'seas_pacf', 'stability', 'diff2x_pacf5', 'trend'],
      dtype='object')

In [33]:
tsf_ = tsf_[fforma_models[0].order_ts_features]

In [34]:
tsf_.head()

,seas_pacf,e_acf10,diff2x_pacf5,diff2_acf1,series_length,diff2_acf10,stability,diff1_acf1,x_acf1,seasonal_period,...,crossing_points,e_acf1,seas_acf1,curvature,diff1_acf10,spike,nperiods,frequency,x_acf10,arch_lm
0,-0.094072,1.461043,1.939661,-0.704965,50,1.658390,0.145407,-0.597489,-0.140900,1,...,30,-0.314732,0.087702,2.184292,1.492544,0.000163,True,12,1.192757,0.010707
1,-0.125610,1.323410,1.555474,-0.705019,50,1.852565,0.106734,-0.521148,0.022792,1,...,25,-0.171970,-0.051201,1.952335,1.458309,0.000522,True,12,1.151826,0.007047
2,-0.068389,1.433410,1.729058,-0.684925,50,1.717888,0.311000,-0.549118,0.236934,1,...,17,-0.397724,-0.079720,0.193432,1.503367,0.000101,True,12,1.400377,0.015278
3,0.066404,1.310671,1.511800,-0.749292,50,2.043407,0.418027,-0.597777,0.448693,1,...,19,-0.321346,0.031309,-0.962357,1.638475,0.000029,True,12,1.873614,0.001879
4,0.117639,1.100052,1.739054,-0.659246,50,1.550988,0.206074,-0.502830,0.172767,1,...,25,-0.227677,-0.035936,-0.057747,1.330616,0.000058,True,12,1.253645,0.003241


In [38]:
fforma_models[0].order_ts_features

Index(['crossing_points', 'seas_acf1', 'diff1_acf10', 'x_pacf5', 'garch_r2',
       'arch_acf', 'lumpiness', 'garch_acf', 'e_acf1', 'diff2_acf10',
       'curvature', 'stability', 'flat_spots', 'seas_pacf', 'trend', 'arch_lm',
       'nperiods', 'linearity', 'arch_2', 'series_length', 'diff2x_pacf5',
       'seasonal_period', 'diff2_acf1', 'diff1x_pacf5', 'spike', 'e_acf10',
       'x_acf1', 'frequency', 'entropy', 'x_acf10', 'diff1_acf1'],
      dtype='object')

In [41]:
fforma_models[0].xgb.feature_names

['crossing_points',
 'seas_acf1',
 'diff1_acf10',
 'x_pacf5',
 'garch_r2',
 'arch_acf',
 'lumpiness',
 'garch_acf',
 'e_acf1',
 'diff2_acf10',
 'curvature',
 'stability',
 'flat_spots',
 'seas_pacf',
 'trend',
 'arch_lm',
 'nperiods',
 'linearity',
 'arch_2',
 'series_length',
 'diff2x_pacf5',
 'seasonal_period',
 'diff2_acf1',
 'diff1x_pacf5',
 'spike',
 'e_acf10',
 'x_acf1',
 'frequency',
 'entropy',
 'x_acf10',
 'diff1_acf1']

In [27]:
?fforma_models[0].predict

Object `fforma_models[0].predict` not found.
